# 【ハンデ計算用ページ CSV出力用】

## ☆ハンデの計算方法

### （新）ペリア方式

- 今回はコンペでよく採用される「新ペリア方式」を採用する。計算方法は以下の通り。
        
\begin{equation*}
 HDCP =  （Score - Par）   × 1.2
\end{equation*}

- 記号は以下の通り。
    - $HDCP$：ハンデ
    - $Score$：ランダムに選出された$6$（$12$）ホールのスコアの合計  &ensp;　&ensp;※（）内の数字は新ペリア方式　以下同様
    - $Par$：ランダムに選出された$6$（$12$）ホールのParの合計
    - $HDCP$が$0$以下の場合は$0$になる。
    
- 補足
    - $HDCP$が同じ場合はGROSS（$Score$）で比較。それでも同点の場合は同率とする。賞金は山分け、景品の場合はジャンケン。 

### ⓪ファイルパス、モード設定（以下、実行するには、ソースコードを選択して「Ctrl」+ 「Enter」）

- スコアシートのファイルパスを設定。ここだけ設定すればあとはテンプレートに入力してスコアを入力するだけで実行可能！

In [1]:
# ファイルパスをここで指定
import pandas as pd
import random
import bisect
import numpy as np

#スコアを集計したCSVのパス
file_path = 'sample.csv'

#最終結果を出力するフォルダのパス
output_path = ''

# モード
# ①ペリア方式か新ペリア方式か（ペリアなら6、新ペリアなら12）
peria = 12

# ②乱数のシード（好きな数字を選ばせることで、それによってホールを決定する。いわば擬似乱数。0 ~ 2^32 -1）
random_seed = 42



### ①スコアシート読込部

- 注意！
    - 以降、ソースコードは絶対に変更しないでください。動かなくなる恐れがあります。
    - ファイルパス要確認

In [2]:
score_sheet = pd.read_csv(file_path)
print("集計スコア")
score_sheet

集計スコア


,Name,HOLE1,HOLE2,HOLE3,HOLE4,HOLE5,HOLE6,HOLE7,HOLE8,HOLE9,HOLE10,HOLE11,HOLE12,HOLE13,HOLE14,HOLE15,HOLE16,HOLE17,HOLE18
0,PAR,4,4,3,4,4,4,4,5,3,4,3,4,4,5,3,4,4,5
1,A,5,6,3,6,6,5,7,6,8,4,5,6,3,7,8,6,5,8
2,B,4,7,4,5,6,4,7,7,5,4,3,6,9,7,8,3,7,9
3,C,4,6,5,8,6,4,4,4,5,6,6,5,6,3,7,4,5,7
4,D,5,6,6,5,6,4,7,8,8,5,6,3,9,7,3,6,7,9
5,E,4,7,5,7,5,3,3,5,5,8,6,6,4,6,7,5,8,7
6,F,5,7,5,4,3,9,7,4,4,4,6,6,9,7,6,5,5,8
7,G,5,6,4,6,6,3,4,4,3,6,6,6,3,4,4,6,7,4


In [3]:
#OUT、IN、GROSSの確認用
name_list = []
out_sum = []
in_sum = []
#gross = []

for i in range(1, len(score_sheet)):
    name_list.append(score_sheet["Name"][i])
    
    out_score = 0
    for j in range(1, 10):
        out_score = out_score + score_sheet['HOLE' + str(j)][i]
    out_sum.append(out_score)
        
    in_score = 0
    for j in range(10, 19):
        in_score = in_score + score_sheet['HOLE' + str(j)][i]
    in_sum.append(in_score)
    
result_df = pd.DataFrame(
    {
        "Name" : name_list,
        "OUT" : out_sum,
        "IN" : in_sum
    }
)
result_df["GROSS"] = result_df["OUT"] + result_df["IN"]
result_df

,Name,OUT,IN,GROSS
0,A,52,52,104
1,B,49,56,105
2,C,46,49,95
3,D,55,55,110
4,E,44,57,101
5,F,48,56,104
6,G,41,46,87


### ②隠しホールの決定

- 隠しホール（ハンデを加味するホール）を以下のルールに則って12ホール選択する。
    - Par4ホールから計8ホール
    - Par3、Par5からそれぞれ2ホールずつ計4ホール
    
    
- 注意！
    - ソースコードは絶対に変更しないでください。動かなくなる恐れがあります。

In [4]:
import random
import bisect

# シードを元にホールが決定される。
np.random.seed(random_seed)

hole = []
hidden_hole = []
c = 0

for i in range(1, 19):
    if score_sheet['HOLE' + str(i)][0] == 3:
        hole.append(i)
        
    elif score_sheet['HOLE' + str(i)][0] == 4:
        hole.append(i)
    
    elif score_sheet['HOLE' + str(i)][0] == 5:
        hole.append(i)

print("ハンデ算出対象に選ばれた" + str(peria) + "つのコースは・・・")

while c < peria:
    number = np.random.choice(hole)
    bisect.insort_left(hidden_hole, number)
    hole.remove(number)
    c = c+1
    
else:
    print(hidden_hole)
    print("に決定！！！")

ハンデ算出対象に選ばれた12つのコースは・・・
[3, 4, 5, 6, 7, 9, 10, 12, 13, 15, 16, 17]
に決定！！！


### ③ハンデの算出

- ハンデ算出方法（再掲）
\begin{equation*}
 HDCP =  （Score - Par）   × 1.2
\end{equation*}

- 記号は以下の通り。
    - $HDCP$：ハンデ
    - $Score$：ランダムに選出された12ホールのスコアの合計
    - $Par$：ランダムに選出された12ホールのParの合計
    - $HDCP$が$0$以下の場合は$0$になる。
    
    
- 注意！
    - ソースコードは絶対に変更しないでください。動かなくなる恐れがあります。

In [5]:
hdcp = []
#各人のHDCPの計算
for i in range(1, len(score_sheet)):
    hd = 0
    for j in range(len(hidden_hole)):
        hd = hd + score_sheet['HOLE' + str(hidden_hole[j])][i] - score_sheet['HOLE' + str(hidden_hole[j])][0]
    if hd < 0:
        hd = 0
    hd = round(hd * 1.2, 1)
    print(score_sheet["Name"][i] + " の HDCPは " + str(hd))
    hdcp.append(hd)

A の HDCPは 26.4
B の HDCPは 27.6
C の HDCPは 24.0
D の HDCPは 28.8
E の HDCPは 25.2
F の HDCPは 26.4
G の HDCPは 15.6


### ④総合結果csvの作成

- ハンデを計算した後に、csvを出力する。


- 注意！
    - ソースコードは絶対に変更しないでください。動かなくなる恐れがあります。


In [6]:
result_df["HDCP"] = hdcp
result_df["NET"] = result_df["GROSS"] - result_df["HDCP"]
sort_result_df = result_df.sort_values(by=["NET", "GROSS"])
sort_result_df["RANK"] = np.arange(1, len(result_df) + 1, 1)
final_result_df = sort_result_df.loc[:, ["RANK", "Name", "OUT", "IN", "GROSS", "HDCP", "NET"]].set_index("RANK")
final_result_df.to_csv(output_path + 'final_result.csv')

In [7]:
print("最終結果")
final_result_df

最終結果


,Name,OUT,IN,GROSS,HDCP,NET
RANK,,,,,,
1,C,46,49,95,24.0,71.0
2,G,41,46,87,15.6,71.4
3,E,44,57,101,25.2,75.8
4,B,49,56,105,27.6,77.4
5,A,52,52,104,26.4,77.6
6,F,48,56,104,26.4,77.6
7,D,55,55,110,28.8,81.2
